# Simulation Study for Noise Filtering

This is the v2 for the simulation study on the sparse jump model comparison with HMM, to show that SJM is able to filter away noisy data by using the weighting in the algorithm.


In [35]:
#Load Packages
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM  # Import GaussianHMM
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from scipy.optimize import linear_sum_assignment
from jumpmodels.sparse_jump import SparseJumpModel
from jumpmodels.jump import JumpModel
from scipy import stats
from joblib import Parallel, delayed
import multiprocessing
from scipy.stats import wilcoxon
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans


## 1. Data Simulation & Utility Functions
def simulate_data(T, P, mu, random_state=None): """ Simulate data from a 2-state Gaussian HMM with correlation (tbd) between the features

In [36]:
# 1. Data simulation
def simulate_data(T, P, mu, random_state=None):
    """
    Simulate data from a 2-state Gaussian HMM with correlated noise for features beyond
    the first 15 informative features.
    """
    rng = np.random.default_rng(random_state)
    
    # Transition matrix for 2 states
    transmat = np.array([[0.9979, 0.0021],
                         [0.0120, 0.9880]])
    
    # Compute stationary distribution
    eigvals, eigvecs = np.linalg.eig(transmat.T)
    stat = np.real(eigvecs[:, np.isclose(eigvals, 1)])
    stat = stat[:, 0]
    stat = stat / np.sum(stat)
    
    # Generate state sequence
    states = np.zeros(T, dtype=int)
    states[0] = rng.choice(np.arange(2), p=stat)
    for t in range(1, T):
        states[t] = rng.choice(np.arange(2), p=transmat[states[t-1]])
    
    # Define means for each state: first 15 features are informative
    means = np.zeros((2, P))
    if P >= 15:
        means[0, :15] = mu
        means[1, :15] = -mu
    else:
        means[0, :P] = mu
        means[1, :P] = -mu
    
    # Prepare correlated noise for features beyond the first 15
    informative = 15
    if P > informative:
        num_noise = P - informative
        sigma = np.full((num_noise, num_noise), 0.213)
        np.fill_diagonal(sigma, 1.0)
        C = np.linalg.cholesky(sigma)
    else:
        C = None
    
    # Generate observations
    X = np.zeros((T, P))
    for t in range(T):
        n_inf = min(P, informative)
        X[t, :n_inf] = rng.normal(loc=means[states[t], :n_inf], scale=1.0, size=n_inf)
        if P > informative:
            noise_indep = rng.normal(loc=0.0, scale=1.0, size=P - informative)
            X[t, informative:] = C @ noise_indep
    return X, states

## 2.Aligning Predicted Labels With True Labels using the Hungarian Algorithm

In [37]:
def align_labels(true_labels, pred_labels):
    """
    Align predicted labels with true labels using the Hungarian algorithm.
    """
    D = confusion_matrix(true_labels, pred_labels)
    row_ind, col_ind = linear_sum_assignment(-D)
    mapping = {col: row for row, col in zip(row_ind, col_ind)}
    aligned = np.array([mapping[x] for x in pred_labels])
    return aligned

## 3. Setting up the function to calcuate the BAC

In [38]:
def calculate_bac(true_states, pred_states):
    """
    Compute balanced accuracy score after aligning predicted state labels.
    """
    aligned_pred = align_labels(true_states, pred_states)
    return balanced_accuracy_score(true_states, aligned_pred)

## 3.1 Computing per state accuracy and calculating statistic

In [39]:
def compute_per_state_accuracy(true_states, pred_states):
    """
    Compute per-state accuracy and BAC.
    Returns:
        acc1: accuracy for state 0,
        acc2: accuracy for state 1,
        bac: balanced accuracy.
    """
    aligned = align_labels(true_states, pred_states)
    cm = confusion_matrix(true_states, aligned, labels=[0, 1])
    acc1 = cm[0, 0] / cm[0].sum() if cm[0].sum() > 0 else np.nan
    acc2 = cm[1, 1] / cm[1].sum() if cm[1].sum() > 0 else np.nan
    bac = (acc1 + acc2) / 2
    return acc1, acc2, bac

def compute_state_statistics(X, true_states, pred_states):
    """
    Compute mean for each state.
    Returns a dict with keys: state0_mean, state1_mean.
    """
    aligned = align_labels(true_states, pred_states)
    stats_dict = {}
    for state in [0, 1]:
        idx = np.where(aligned == state)[0]
        if len(idx) > 0:
            stats_dict[f'state{state}_mean'] = np.mean(X[idx])
        else:
            stats_dict[f'state{state}_mean'] = np.nan
    return stats_dict

## 3.2 Computing transition probabilities

In [40]:
def compute_transition_probabilities(state_seq):
    """
    Compute q12 and q21 from the state sequence.
    q12: transitions from state 0 to 1 / total transitions from state 0.
    q21: transitions from state 1 to 0 / total transitions from state 1.
    """
    state_seq = np.array(state_seq)
    prev = state_seq[:-1]
    nxt = state_seq[1:]
    if np.sum(prev==0) > 0:
        q12 = np.sum((prev==0) & (nxt==1)) / np.sum(prev==0)
    else:
        q12 = np.nan
    if np.sum(prev==1) > 0:
        q21 = np.sum((prev==1) & (nxt==0)) / np.sum(prev==1)
    else:
        q21 = np.nan
    return q12, q21

## 3.3 Standardizing features

In [41]:
def standardize_features(X):
    """
    Standardize features using the same procedure as in study 2.
    The function converts X into a DataFrame, applies DataClipperStd with mul=3.0,
    then applies StandardScalerPD (zero mean, unit variance) and returns the standardized array.
    """
    X_df = pd.DataFrame(X)
    clipper = DataClipperStd(mul=3.0)
    scaler = StandardScalerPD()
    X_clipped = clipper.fit_transform(X_df)
    X_scaled = scaler.fit_transform(X_clipped)
    return X_scaled.values

## 4. Functions for model formulation

### 4.1 HMM With Nystrup (2020b) initialization

In [42]:
def run_mle(observations, n_components=2, init_type='default', seed=None):
    """
    Fit a Gaussian HMM to the data.
    init_type: 'default' uses fixed initialization; 'kmeans' uses KMeans initialization.
    For 'default', we use:
      - startprob_ = [1, 0]
      - transmat_ = [[0.9, 0.1], [0.1, 0.9]]
      - Means initialized to [0, 0] and covars to a small positive value.
    """
    model = GaussianHMM(n_components=n_components, covariance_type='diag', n_iter=100, random_state=seed)
    
    if init_type=='default':
        model.startprob_ = np.array([1.0, 0.0])
        model.transmat_ = np.array([[0.9, 0.1],
                                    [0.1, 0.9]])
        model.means_ = np.zeros((n_components, observations.shape[1]))
        # Ensure nonzero covariances:
        model.covars_ = np.full((n_components, observations.shape[1]), 1e-2)
        model.init_params = ''  # do not reinitialize
    elif init_type=='kmeans':
        kmeans = KMeans(n_clusters=n_components, n_init=10, random_state=seed)
        labels = kmeans.fit_predict(observations)
        means = []
        covars = []
        for i in range(n_components):
            obs_i = observations[labels==i]
            means.append(np.mean(obs_i, axis=0))
            covars.append(np.var(obs_i, axis=0) + 1e-2)
        model.startprob_ = np.ones(n_components) / n_components
        model.transmat_ = np.ones((n_components, n_components)) / n_components
        model.means_ = np.array(means)
        model.covars_ = np.array(covars)
        model.init_params = 'tmc'
    model.fit(observations)
    pred_states = model.predict(observations)
    return model, pred_states

# Convenience wrappers:
def run_mle_default(observations, seed=None):
    return run_mle(observations, init_type='default', seed=seed)

def run_mle_kmeans(observations, seed=None):
    return run_mle(observations, init_type='kmeans', seed=seed)

### 4.2 Normal (Standard) Jump Model with Grid Search over λ

In [43]:
def run_jump_model_grid_search(X, true_states, n_components=2, random_state=None):
    """
    Grid search for the jump model over a range of lambda values.
    Returns best predicted states, best BAC, best lambda, and a DataFrame of grid search results.
    """
    lambda_values = np.logspace(-2, 4, 1)
    best_bac = -np.inf
    best_labels = None
    best_lambda = None
    grid_results = []
    for lam in lambda_values:
        model = JumpModel(n_components=n_components, jump_penalty=lam, cont=False,
                          max_iter=10, random_state=random_state)
        model.fit(X)
        labels = model.labels_
        bac = calculate_bac(true_states, labels)
        grid_results.append({'lambda': lam, 'BAC': bac})
        if bac > best_bac:
            best_bac = bac
            best_labels = labels
            best_lambda = lam
    return best_labels, best_bac, best_lambda, pd.DataFrame(grid_results)

### 4.3 Sparse Jump Model with Grid Search over λ and kappa

In [44]:
def run_sparse_jump_model_grid_search(X, true_states, n_components=2, random_state=None):
    """
    Grid search for the sparse jump model over combinations of lambda and kappa.
    Returns best predicted states, best BAC, best lambda, best kappa, and a DataFrame of grid search results.
    """
    lambdas = np.logspace(-1, 2, 1)
    p = X.shape[1]
    kappas = np.linspace(1, np.sqrt(p), 1)
    best_bac = -np.inf
    best_labels = None
    best_lambda = None
    best_kappa = None
    grid_results = []
    for lam in lambdas:
        for kappa in kappas:
            max_feats = kappa**2
            model = SparseJumpModel(n_components=n_components, jump_penalty=lam, cont=False,
                                     max_feats=max_feats, max_iter=10, random_state=random_state)
            model.fit(X)
            labels = model.labels_
            bac = calculate_bac(true_states, labels)
            grid_results.append({'lambda': lam, 'kappa': kappa, 'max_feats': max_feats, 'BAC': bac})
            if bac > best_bac:
                best_bac = bac
                best_labels = labels
                best_lambda = lam
                best_kappa = kappa
    return best_labels, best_bac, best_lambda, best_kappa, pd.DataFrame(grid_results)

# 5. Main Execution
 We split the code into three sections for each model and then combine results at the end.


In [45]:
# Settings
T = 500
mu_values = [0.02, 0.05, 0.1, 0.25, 0.5]
p_values = [15, 30, 60, 150, 300]
n_simulations = 10  # adjust as needed
n_components = 2

# Lists to collect simulation details and grid search results
simulation_results = []
grid_search_jump_list = []
grid_search_sparse_list = []

num_cores = multiprocessing.cpu_count()

for mu in mu_values:
    for P in p_values:
        sim_results = []
        # Parallelize simulations for this mu and P combination
        results = Parallel(n_jobs=num_cores)(
            delayed(lambda sim: {
                'sim': sim,
                'X': simulate_data(T, P, mu, random_state=sim)[0],
                'true_states': simulate_data(T, P, mu, random_state=sim)[1]
            })(sim) for sim in range(n_simulations)
        )
        for res in results:
            X = res['X']
            true_states = res['true_states']
            
            # --- MLE default and MLE_k (kmeans) on raw data ---
            model_mle, pred_mle = run_mle_default(X, seed=res['sim'])
            model_mle_k, pred_mle_k = run_mle_kmeans(X, seed=res['sim'])
            bac_mle = calculate_bac(true_states, pred_mle)
            bac_mle_k = calculate_bac(true_states, pred_mle_k)
            
            # --- Standardize features for jump models ---
            X_std = standardize_features(X)
            
            # --- Jump model grid search ---
            pred_jump, bac_jump, best_lambda_jump, df_grid_jump = run_jump_model_grid_search(
                X_std, true_states, n_components=n_components, random_state=res['sim'])
            
            # --- Sparse jump model grid search ---
            pred_sparse, bac_sparse, best_lambda_sparse, best_kappa_sparse, df_grid_sparse = run_sparse_jump_model_grid_search(
                X_std, true_states, n_components=n_components, random_state=res['sim'])
            
            # --- Compute performance metrics ---
            acc1_mle, acc2_mle, _ = compute_per_state_accuracy(true_states, pred_mle)
            acc1_mle_k, acc2_mle_k, _ = compute_per_state_accuracy(true_states, pred_mle_k)
            acc1_jump, acc2_jump, _ = compute_per_state_accuracy(true_states, pred_jump)
            acc1_sparse, acc2_sparse, _ = compute_per_state_accuracy(true_states, pred_sparse)
            
            stats_mle = compute_state_statistics(X, true_states, pred_mle)
            stats_mle_k = compute_state_statistics(X, true_states, pred_mle_k)
            stats_jump = compute_state_statistics(X, true_states, pred_jump)
            stats_sparse = compute_state_statistics(X, true_states, pred_sparse)
            
            q12_mle, q21_mle = compute_transition_probabilities(pred_mle)
            q12_mle_k, q21_mle_k = compute_transition_probabilities(pred_mle_k)
            q12_jump, q21_jump = compute_transition_probabilities(pred_jump)
            q12_sparse, q21_sparse = compute_transition_probabilities(pred_sparse)
            
            # Save all results in a dict for this simulation run (volatility fields removed)
            run_dict = {
                'mu': mu,
                'P': P,
                'sim': res['sim'],
                # MLE (default)
                'MLE_BAC': bac_mle,
                'MLE_acc1': acc1_mle,
                'MLE_acc2': acc2_mle,
                'MLE_state0_mean': stats_mle.get('state0_mean', np.nan),
                'MLE_state1_mean': stats_mle.get('state1_mean', np.nan),
                'MLE_q12': q12_mle,
                'MLE_q21': q21_mle,
                # MLE_k (kmeans)
                'MLEK_BAC': bac_mle_k,
                'MLEK_acc1': acc1_mle_k,
                'MLEK_acc2': acc2_mle_k,
                'MLEK_state0_mean': stats_mle_k.get('state0_mean', np.nan),
                'MLEK_state1_mean': stats_mle_k.get('state1_mean', np.nan),
                'MLEK_q12': q12_mle_k,
                'MLEK_q21': q21_mle_k,
                # Jump Model
                'Jump_BAC': bac_jump,
                'Jump_acc1': acc1_jump,
                'Jump_acc2': acc2_jump,
                'Jump_state0_mean': stats_jump.get('state0_mean', np.nan),
                'Jump_state1_mean': stats_jump.get('state1_mean', np.nan),
                'Jump_q12': q12_jump,
                'Jump_q21': q21_jump,
                'Jump_best_lambda': best_lambda_jump,
                # Sparse Jump Model
                'SparseJump_BAC': bac_sparse,
                'SparseJump_acc1': acc1_sparse,
                'SparseJump_acc2': acc2_sparse,
                'SparseJump_state0_mean': stats_sparse.get('state0_mean', np.nan),
                'SparseJump_state1_mean': stats_sparse.get('state1_mean', np.nan),
                'SparseJump_q12': q12_sparse,
                'SparseJump_q21': q21_sparse,
                'SparseJump_best_lambda': best_lambda_sparse,
                'SparseJump_best_kappa': best_kappa_sparse
            }
            sim_results.append(run_dict)
            
            # Save grid search results with additional identifiers
            df_grid_jump['mu'] = mu
            df_grid_jump['P'] = P
            df_grid_jump['sim'] = res['sim']
            grid_search_jump_list.append(df_grid_jump)
            
            df_grid_sparse['mu'] = mu
            df_grid_sparse['P'] = P
            df_grid_sparse['sim'] = res['sim']
            grid_search_sparse_list.append(df_grid_sparse)
            
        simulation_results.extend(sim_results)

# Final DataFrames
df_simulation = pd.DataFrame(simulation_results)
df_grid_jump = pd.concat(grid_search_jump_list, ignore_index=True)
df_grid_sparse = pd.concat(grid_search_sparse_list, ignore_index=True)

print("Simulation results shape:", df_simulation.shape)
print("Jump grid search results shape:", df_grid_jump.shape)
print("Sparse Jump grid search results shape:", df_grid_sparse.shape)

Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in 

Simulation results shape: (250, 34)
Jump grid search results shape: (250, 5)
Sparse Jump grid search results shape: (250, 7)


## 6 Wilcoxon Test Table Generation

In [46]:
# Create a Wilcoxon table comparing Sparse Jump BAC versus the other models (MLE, MLEK, Jump)
wilcoxon_rows = []

for mu in mu_values:
    for P in p_values:
        df_subset = df_simulation[(df_simulation['mu'] == mu) & (df_simulation['P'] == P)]
        if len(df_subset) < 2:
            continue
        # Compare BAC: Sparse Jump vs MLE
        stat, p_sparse_mle = wilcoxon(df_subset['SparseJump_BAC'], df_subset['MLE_BAC'])
        # Compare BAC: Sparse Jump vs MLE_k
        stat, p_sparse_mlek = wilcoxon(df_subset['SparseJump_BAC'], df_subset['MLEK_BAC'])
        # Compare BAC: Sparse Jump vs Jump
        stat, p_sparse_jump = wilcoxon(df_subset['SparseJump_BAC'], df_subset['Jump_BAC'])
        
        wilcoxon_rows.append({
            'mu': mu,
            'P': P,
            'p_value_SparseJump_vs_MLE': p_sparse_mle,
            'p_value_SparseJump_vs_MLEK': p_sparse_mlek,
            'p_value_SparseJump_vs_Jump': p_sparse_jump
        })

df_wilcoxon = pd.DataFrame(wilcoxon_rows)
print("Wilcoxon test results (Sparse Jump vs other models):")
print(df_wilcoxon)


Wilcoxon test results (Sparse Jump vs other models):
      mu    P  p_value_SparseJump_vs_MLE  p_value_SparseJump_vs_MLEK  p_value_SparseJump_vs_Jump
0   0.02   15                   0.556641                    0.232422                    0.921875
1   0.02   30                   0.625000                    0.769531                    0.845703
2   0.02   60                   0.275391                    0.275391                    0.375000
3   0.02  150                   1.000000                    0.845703                    0.845703
4   0.02  300                   0.048828                    0.260393                    0.232422
5   0.05   15                   0.160156                    0.769531                    1.000000
6   0.05   30                   0.695312                    0.845703                    1.000000
7   0.05   60                   0.232422                    0.232422                    0.375000
8   0.05  150                   1.000000                    0.845703      

### 7 Summary Tables

In [47]:
# Summary statistics of the df_simulation dataframe without transition probabilities
def summarize_method(group, prefix):
    """
    Extracts the mean of relevant columns for a given method prefix 
    (e.g., 'MLE', 'MLEK', 'Jump', 'SparseJump'), excluding transition probabilities.
    """
    return {
        'state0_mean': group[f'{prefix}_state0_mean'].mean(),
        'state1_mean': group[f'{prefix}_state1_mean'].mean(),
        'acc1':        group[f'{prefix}_acc1'].mean(),
        'acc2':        group[f'{prefix}_acc2'].mean(),
        'BAC':         group[f'{prefix}_BAC'].mean()
    }

# Group the data by mu, P
grouped = df_simulation.groupby(['mu', 'P'])

# Loop over each mu-P combination
for (m, p), group in grouped:
    # Summaries for each model
    row_mle   = summarize_method(group, 'MLE')
    row_mlek  = summarize_method(group, 'MLEK')
    row_jump  = summarize_method(group, 'Jump')
    row_sjump = summarize_method(group, 'SparseJump')
    
    print(f"\nSummary for mu={m} & P={p}")
    print("--------------------------------------------------------------------------")
    print("{:<12s}  {:>12s}  {:>12s}  {:>12s}  {:>12s}  {:>12s}".format(
        "", "state0_mean", "state1_mean", "acc1", "acc2", "BAC"
    ))
    
    def fmt_row(method_name, r):
        return "{:<12s}  {:>12.4f}  {:>12.4f}  {:>12.4f}  {:>12.4f}  {:>12.4f}".format(
            method_name,
            r['state0_mean'], r['state1_mean'], 
            r['acc1'], r['acc2'], r['BAC']
        )
    
    print(fmt_row("MLE def", row_mle))
    print(fmt_row("MLE k-means", row_mlek))
    print(fmt_row("Jump Model", row_jump))
    print(fmt_row("Sparse Jump", row_sjump))


Summary for mu=0.02 & P=15
--------------------------------------------------------------------------
               state0_mean   state1_mean          acc1          acc2           BAC
MLE def             0.0178        0.0424        0.7548        0.3302        0.5578
MLE k-means         0.0193        0.0126        0.6772        0.3150        0.5080
Jump Model          0.0288        0.0137        0.5087        0.5497        0.5270
Sparse Jump         0.0383       -0.0010        0.5389        0.5109        0.5268

Summary for mu=0.02 & P=30
--------------------------------------------------------------------------
               state0_mean   state1_mean          acc1          acc2           BAC
MLE def            -0.0977        0.1446        0.5646        0.4307        0.5058
MLE k-means        -0.0973        0.1450        0.5678        0.4276        0.5060
Jump Model         -0.0249        0.0569        0.5401        0.4681        0.5084
Sparse Jump         0.0530       -0.0381       

### 7.1 statistical testing summary:

In [48]:
def recalc_true_metrics(P, mu, sim, T=500):
    """
    For a given simulation run with number of features P, true mean mu,
    simulation id sim, and fixed length T (default=500),
    simulate the data, decode using the true parameters, and compute
    the per-state accuracies and BAC.
    """
    y, true_states = simulate_data(T, P, mu, random_state=sim)
    pred_true = viterbi_true(y)
    return compute_per_state_accuracy(true_states, pred_true)

# Apply the recalculation row-wise to update true metrics in df_simulation.
true_metrics = df_simulation.apply(lambda row: recalc_true_metrics(row['P'], row['mu'], row['sim']), axis=1)
df_simulation['True_acc1'] = true_metrics.apply(lambda x: x[0])
df_simulation['True_acc2'] = true_metrics.apply(lambda x: x[1])
df_simulation['True_BAC']  = true_metrics.apply(lambda x: x[2])

# Optionally, print a sample to verify:
print("Sample recalculated true metrics:")
print(df_simulation[['sim', 'mu', 'P', 'True_acc1', 'True_acc2', 'True_BAC']].head())


TypeError: SeedSequence expects int or sequence of ints for entropy not 0.0

In [ ]:
#Summary statistics

# Define the metrics for which we want to create error values.
# In this simulation, we use:
#   - state0_mean: true value is mu
#   - state1_mean: true value is -mu
#   - acc1, acc2, and BAC: we assume the ideal is 1.0 (i.e. perfect classification)
metric_list = ["state0_mean", "state1_mean", "acc1", "acc2", "BAC"]

# We also list the model prefixes we want to compare.
model_prefixes = ["MLEK", "Jump", "SparseJump"]

# Create a copy of df_simulation so we can add error columns.
# (Assume df_simulation is your main results DataFrame.)
final_result_errors = df_simulation.copy()

# For each row in the DataFrame, set the appropriate reference values.
# Here, the true reference for state0_mean and state1_mean depend on mu, 
# while the reference for acc1, acc2, and BAC is 1.0.
def compute_ref_values(row):
    mu = row['mu']
    return {
        "state0_mean": mu,
        "state1_mean": -mu,
        "acc1": 0.992246,
        "acc2": 0.921367,
        "BAC": 0.956806
    }

# Apply the reference computation row-wise (storing as a Series of dictionaries)
refs = final_result_errors.apply(compute_ref_values, axis=1)

# Now create error columns for each metric and each model prefix.
for metric in metric_list:
    for prefix in model_prefixes:
        error_col = f"{prefix}_{metric}_error"
        # Compute the absolute error relative to the reference value in each row.
        final_result_errors[error_col] = final_result_errors.apply(
            lambda row: abs(row[f"{prefix}_{metric}"] - compute_ref_values(row)[metric]),
            axis=1
        )

# Next, we group the data by 'mu' and 'P' (as in your summary table)
grouped = final_result_errors.groupby(['mu', 'P'])

# Define safe wrappers for Wilcoxon tests.
def two_sided_wilcoxon_p(diffs, atol=1e-10):
    diffs = np.array(diffs)
    diffs = diffs[~np.isnan(diffs)]
    if len(diffs) == 0:
        return 1.0
    if np.allclose(diffs, 0, atol=atol):
        return 1.0
    try:
        _, p = wilcoxon(diffs)
        return p
    except ValueError:
        return np.nan

def one_sided_pval(diffs, p_two_sided):
    if np.isnan(p_two_sided):
        return np.nan
    mean_diff = np.mean(diffs)
    if mean_diff > 0:
        return p_two_sided / 2
    else:
        return 1 - p_two_sided / 2

# For each group, we will calculate the Wilcoxon test p-values comparing the errors.
# We compare the following for each metric:
#   1. Difference between Jump and SparseJump errors.
#   2. Difference between MLEK and Jump errors.
#   3. Difference between MLEK and SparseJump errors.
for (mu_val, P_val), group in grouped:
    table_rows = []
    for metric in metric_list:
        col_mlek = f"MLEK_{metric}_error"
        col_jump = f"Jump_{metric}_error"
        col_sparse = f"SparseJump_{metric}_error"
        
        # Extract the error series from the group.
        e_mlek = group[col_mlek]
        e_jump = group[col_jump]
        e_sparse = group[col_sparse]
        
        # Compute mean errors for reporting.
        mean_mlek = e_mlek.mean()
        mean_jump = e_jump.mean()
        mean_sparse = e_sparse.mean()
        
        # 1) Test: SparseJump vs. Jump (we test if SparseJump error is lower than Jump error)
        diff_sj = e_jump - e_sparse  # positive differences indicate Jump error > SparseJump error
        p2_sj = two_sided_wilcoxon_p(diff_sj)
        p1_sj = one_sided_pval(diff_sj, p2_sj)
        
        # 2) Test: Jump vs. MLEK (we test if Jump error is lower than MLEK error)
        diff_jm = e_mlek - e_jump
        p2_jm = two_sided_wilcoxon_p(diff_jm)
        p1_jm = one_sided_pval(diff_jm, p2_jm)
        
        # 3) Test: SparseJump vs. MLEK (we test if SparseJump error is lower than MLEK error)
        diff_sm = e_mlek - e_sparse
        p2_sm = two_sided_wilcoxon_p(diff_sm)
        p1_sm = one_sided_pval(diff_sm, p2_sm)
        
        table_rows.append({
            "Metric": metric,
            "MLEK Mean Error": mean_mlek,
            "Jump Mean Error": mean_jump,
            "SparseJump Mean Error": mean_sparse,
            "p (Sparse < Jump)": p1_sj,
            "p (Jump < MLEK)": p1_jm,
            "p (Sparse < MLEK)": p1_sm
        })
    
    wilcoxon_table = pd.DataFrame(table_rows)
    print(f"\n=== Wilcoxon Test Results for mu = {mu_val} and P = {P_val} ===")
    print(wilcoxon_table.to_string(index=False, float_format='%.4f'))

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/s


=== Wilcoxon Test Results for mu = 0.02 and P = 15 ===
     Metric  MLEK Mean Error  Jump Mean Error  SparseJump Mean Error  p (Sparse < Jump)  p (Jump < MLEK)  p (Sparse < MLEK)
state0_mean           0.0360           0.0353                 0.0402             0.8698           0.3848             0.5771
state1_mean           0.0644           0.0300                 0.0433             0.8086           0.2734             0.4551
       acc1           0.2858           0.3166                 0.3774             0.5705           0.7539             0.8125
       acc2           0.6191           0.4541                 0.3669             0.3897           0.0098             0.0059
        BAC           0.4399           0.3658                 0.3578             0.2573           0.0068             0.0010

=== Wilcoxon Test Results for mu = 0.02 and P = 30 ===
     Metric  MLEK Mean Error  Jump Mean Error  SparseJump Mean Error  p (Sparse < Jump)  p (Jump < MLEK)  p (Sparse < MLEK)
state0_mean         

/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/s